<a href="https://colab.research.google.com/github/vrangayyan6/Machine-Learning/blob/main/DistilBERT_v2_Pan12_full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Based on DistilBERT


# Installing the transformers library

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.1MB 2.9MB/s 
     |████████████████████████████████| 3.0MB 17.5MB/s 
     |████████████████████████████████| 890kB 31.8MB/s 
     |████████████████████████████████| 1.1MB 44.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=91865995909b76904882e0689559100025877da0e2114b1449c0ea2b1a44218f
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers as ppb
from transformers import get_linear_schedule_with_warmup, EvalPrediction
from typing import Dict, Optional
import warnings
warnings.filterwarnings('ignore')


## Importing the dataset

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


# PAN-12 dataset
Using PAN-12 dataset, comment each labeled as either positive (has the value 1) or negative (has the value 0).

In [ ]:
train_neg_path = "/content/drive/My Drive/Omdena/PAN-12pre-processed_data/Train_Neg.csv"
train_neg_df = pd.read_csv(train_neg_path, index_col=0, header=0, names=['id', 'Comments'])
train_neg_df.insert(2, 'label', 0)

train_pos_path = "/content/drive/My Drive/Omdena/PAN-12pre-processed_data/Train_pos.csv"
train_pos_df = pd.read_csv(train_pos_path, index_col=0, header=0, names=['id', 'Comments'])
train_pos_df.insert(2, 'label', 1)

test_neg_path = "/content/drive/My Drive/Omdena/PAN-12pre-processed_data/Test_Neg.csv"
test_neg_df = pd.read_csv(test_neg_path, index_col=0, header=0, names=['id', 'Comments'])
test_neg_df.insert(2, 'label', 0)

test_pos_path = "/content/drive/My Drive/Omdena/PAN-12pre-processed_data/Test_pos.csv"
test_pos_df = pd.read_csv(test_pos_path, index_col=0, header=0, names=['id', 'Comments'])
test_pos_df.insert(2, 'label', 1)

train_df = train_neg_df.append(train_pos_df)
test_df = test_neg_df.append(test_pos_df)


combine_df = train_df.append(test_df)
combine_df = combine_df.drop(columns=['id'])

print(combine_df.shape)
print(combine_df['label'].value_counts())
combine_df.head()

(197937, 2)
0    193943
1      3994
Name: label, dtype: int64


,Comments,label
0,b8810fee2f4a71f849f3f7409546d1d9 - do you have...,0
1,say asl and i&amp;apos;ll rip your heart out.a...,0
2,joint.?.in my hand.sorry.I&amp;apos;m.korea st...,0
3,hey :) male or female? Australian male here.h...,0
4,if you have a channel with ports 1 and 2.and y...,0


In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import csv

stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer("english")

def process_comments(comment):
  word_tokens = []
  if (type(comment)== str):
    word_tokens = [x.lower() for x in comment.split() if x.isalpha()]
  #comment = unicode(comment, 'utf8')
  tokens_stop = [word for word in word_tokens if (word not in stop_words)]
  # tokens_stop = [stemmer.stem(word) for word in word_tokens if (word not in stop_words)]
  comment_updated = []
  if len(tokens_stop) == 0:
    comment_updated = np.NaN
  else:
    comment_updated = ' '.join(tokens_stop)
  return comment_updated

combine_df['Comments'] = combine_df.apply(lambda row: process_comments(row['Comments']), axis=1)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# print( combine_df['label'].apply(lambda l: l if l.isalpha() )
print(combine_df['label'].value_counts())

0    193943
1      3994
Name: label, dtype: int64


In [ ]:
print(combine_df.isnull().sum())

Comments    58184
label           0
dtype: int64


In [ ]:
combine_df = combine_df.dropna()
print(combine_df.isnull().sum())
combine_df = combine_df.reset_index(drop=True)
print(combine_df['label'].value_counts())
print(combine_df.shape)

Comments    0
label       0
dtype: int64
0    135895
1      3858
Name: label, dtype: int64
(139753, 2)


In [ ]:
# combine_df = combine_df.groupby('label').apply(lambda x: x.sample(n=3800, random_state=10))

# max comment length
print('max len ', max(combine_df['Comments'].apply(len)))
combine_df.head()

max len  624687


,Comments,label
0,set test cases syntax also used wf using test ...,0
1,say asl rip heart heart eats,0
2,see see se emy show see,0
3,hey male australian male,0
4,channel ports try send obviously semantics cra...,0


## Loading the Pre-trained model

In [ ]:
# For DistilBERT 'distilbert-base-uncased'
model_class, tokenizer_class, pretrained_weights = (ppb.AutoModelForSequenceClassification, ppb.AutoTokenizer, 'distilbert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

## Creating the Dataset


In [ ]:
train_df, valid_df = train_test_split(combine_df, test_size=0.2)
print(combine_df.shape, train_df.shape, valid_df.shape)

(139753, 2) (111802, 2) (27951, 2)


In [ ]:
train_encodings = tokenizer(train_df.loc[:, "Comments"].to_list(), truncation=True, padding=True)
valid_encodings = tokenizer(valid_df.loc[:, "Comments"].to_list(), truncation=True, padding=True)

In [ ]:
import torch

class OurDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = OurDataset(train_encodings, train_df.loc[:, "label"].to_list())
valid_dataset = OurDataset(valid_encodings, valid_df.loc[:, "label"].to_list())


# Training & Validation

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import EvalPrediction
from typing import Dict, Optional


def compute_metrics(p: EvalPrediction) -> Dict:
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    # preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=500,
    logging_first_step=True,
    save_steps=1000,
    # evaluate_during_training=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
)



In [ ]:
trainer.train()

{'loss': 0.0012608433961868286, 'learning_rate': 1.0000000000000001e-07, 'epoch': 0.00014310246136233542, 'total_flos': 3290972651520, 'step': 1}
{'loss': 0.13885233104228972, 'learning_rate': 5e-05, 'epoch': 0.07155123068116771, 'total_flos': 1645486325760000, 'step': 500}
{'loss': 0.06456243896484375, 'learning_rate': 4.8778342455043e-05, 'epoch': 0.14310246136233543, 'total_flos': 3290972651520000, 'step': 1000}
{'loss': 0.05242762756347656, 'learning_rate': 4.7556684910086006e-05, 'epoch': 0.21465369204350315, 'total_flos': 4936458977280000, 'step': 1500}
{'loss': 0.05530654907226563, 'learning_rate': 4.6335027365129005e-05, 'epoch': 0.28620492272467085, 'total_flos': 6581945303040000, 'step': 2000}
{'loss': 0.05796853637695312, 'learning_rate': 4.511336982017201e-05, 'epoch': 0.3577561534058386, 'total_flos': 8227431628800000, 'step': 2500}
{'loss': 0.06411959838867187, 'learning_rate': 4.3891712275215016e-05, 'epoch': 0.4293073840870063, 'total_flos': 9872917954560000, 'step': 30

{'loss': 0.05430169677734375, 'learning_rate': 3.411845191555903e-05, 'epoch': 1.0017172295363481, 'total_flos': 23035574445895680, 'step': 7000}
{'loss': 0.04310791015625, 'learning_rate': 3.289679437060204e-05, 'epoch': 1.0732684602175158, 'total_flos': 24681060771655680, 'step': 7500}
{'loss': 0.0548165283203125, 'learning_rate': 3.1675136825645036e-05, 'epoch': 1.1448196908986834, 'total_flos': 26326547097415680, 'step': 8000}
{'loss': 0.0661168212890625, 'learning_rate': 3.045347928068804e-05, 'epoch': 1.216370921579851, 'total_flos': 27972033423175680, 'step': 8500}
{'loss': 0.040873291015625, 'learning_rate': 2.923182173573104e-05, 'epoch': 1.287922152261019, 'total_flos': 29617519748935680, 'step': 9000}
{'loss': 0.0458193359375, 'learning_rate': 2.8010164190774042e-05, 'epoch': 1.3594733829421866, 'total_flos': 31263006074695680, 'step': 9500}
{'loss': 0.042478515625, 'learning_rate': 2.6788506645817048e-05, 'epoch': 1.4310246136233542, 'total_flos': 32908492400455680, 'step':

{'loss': 0.0427384033203125, 'learning_rate': 1.7015246286161064e-05, 'epoch': 2.0034344590726962, 'total_flos': 46071148891791360, 'step': 14000}
{'loss': 0.038135009765625, 'learning_rate': 1.5793588741204066e-05, 'epoch': 2.0749856897538637, 'total_flos': 47716635217551360, 'step': 14500}
{'loss': 0.040004638671875, 'learning_rate': 1.4571931196247068e-05, 'epoch': 2.1465369204350315, 'total_flos': 49362121543311360, 'step': 15000}
{'loss': 0.0339853515625, 'learning_rate': 1.3350273651290072e-05, 'epoch': 2.2180881511161994, 'total_flos': 51007607869071360, 'step': 15500}
{'loss': 0.0302108154296875, 'learning_rate': 1.2128616106333074e-05, 'epoch': 2.289639381797367, 'total_flos': 52653094194831360, 'step': 16000}
{'loss': 0.0283154296875, 'learning_rate': 1.0906958561376076e-05, 'epoch': 2.3611906124785347, 'total_flos': 54298580520591360, 'step': 16500}
{'loss': 0.0394898681640625, 'learning_rate': 9.685301016419078e-06, 'epoch': 2.432741843159702, 'total_flos': 5594406684635136

TrainOutput(global_step=20964, training_loss=0.045380372351486474)

In [ ]:
trainer.evaluate()


{'eval_loss': 0.052774768290460954, 'eval_accuracy': 0.9878358556044506, 'eval_f1': 0.7615708274894811, 'eval_precision': 0.8524332810047096, 'eval_recall': 0.688212927756654, 'epoch': 3.0, 'total_flos': 68988248322232320, 'step': 20964}


{'epoch': 3.0,
 'eval_accuracy': 0.9878358556044506,
 'eval_f1': 0.7615708274894811,
 'eval_loss': 0.052774768290460954,
 'eval_precision': 0.8524332810047096,
 'eval_recall': 0.688212927756654,
 'total_flos': 68988248322232320}

In [ ]:
# lower f1 compared to equal dataset